In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
path_to_file = 'shakespeare.txt' 
with open(path_to_file,'r') as f: 
    text = f.read()
    

In [3]:
vocab = sorted(set(text))

In [4]:
char_to_ind = {u:i for i, u in enumerate(vocab)} 
ind_to_char = np.array(vocab) 
encoded_text = np.array([char_to_ind[char] for char in text]) 
encoded_text

array([ 0,  1,  1, ..., 30, 39, 29])

In [5]:
seq_len = 120 
total_num_seq = len(text) // (seq_len + 1)

In [7]:
# creatimg training sequences 
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)
